# MO412A - Project

In [11]:
import requests
from bs4 import BeautifulSoup
import datetime
import pandas as pd
from tqdm import tqdm
import os
from pypdf import PdfReader

In [12]:
def check_string_content(str_input: str):
    banned_starts = [
        "Ementa:",
        "§ 2º do art",
        "1.166, de 2020",
        "3.626/2023.",
        "(art. 51"
    ]
    return str_input.startswith(tuple(banned_starts))

In [13]:
import re
import pandas as pd

def process_page(texto):
    # Expresión regular para extraer filas de la tabla
    regex_filas = r"([\w\sçãéíóúáâêôÇÕÉÍÁ]+)\s+([A-Z]{2})\s+([\w/]+)\s+(.+)"
    # Matchea: Nombre del senador, Estado (UF), Partido, Voto

    # Buscar todas las coincidencias
    rows = []
    for split in texto.split("\n"):
        if len(split.strip()) == 0:
            continue
        coincidencias = re.findall(regex_filas, split)
        if len(coincidencias) > 0:
            # print(coincidencias[0][3])
            if check_string_content(coincidencias[0][3]):
                continue
            row_content = list(coincidencias[0])
            row_content[0] = row_content[0].strip()
            rows.append(row_content)
            # print(rows)
            # assert False
    # print(rows)

    # Crear un DataFrame
    columnas = ["SENADOR", "UF", "PARTIDO", "VOTO"]
    df_votaciones = pd.DataFrame(rows, columns=columnas)
    return df_votaciones


In [14]:
votos = pd.DataFrame()
for file in os.listdir('files'):
    filename = 'files/' + file
    reader = PdfReader(filename)
    for page in reader.pages:
        content = page.extract_text(extraction_mode="layout", layout_mode_space_vertically=False)
        df = process_page(content)
        df['file'] = file
        votos = pd.concat([votos, df])
    

In [15]:
votos = votos[votos['UF'] != 'UF']
votos.to_csv('votos_updated.csv')